In [ ]:


import traceback
from accelerate import Accelerator, InitProcessGroupKwargs
from accelerate.utils import pad_across_processes, broadcast
from collections import defaultdict
from dataclasses import dataclass, field, asdict
from datasets import load_dataset, load_from_disk, DatasetDict, Dataset, concatenate_datasets
from datetime import datetime,  timedelta
import time
from functools import partial
import json
import os
import random
from src.python_engine import run_python_code
from src.utils import set_seed, floatify, compute_ETA
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, get_linear_schedule_with_warmup, AdamW, get_constant_schedule_with_warmup
import wandb
import pandas as pd
import shutil
import signal
from contextlib import contextmanager
from torch.utils.data.distributed import DistributedSampler
from accelerate import notebook_launcher



In [ ]:

tqdm = partial(tqdm, ncols=0, leave=False)


TIMEOUT = 10
instruction=None
cot_trigger=None
answer_trigger=None
def setup_cot(src_name):
    assert src_name in ['gsm8k', 'mathqa', 'svamp', 'mathqa-numeric', 'zhouyi']
    global instruction
    global cot_trigger
    global answer_trigger
    # Complete output is in this form: f'{instruction}{question.strip()}{cot_trigger}{answer_cot.strip()}'
    instruction = 'Question:\n'
    cot_trigger = '\nAnswer reasoning:\n'
    answer_trigger = '\n因此，答案是：'
    return 

post_process_final_answer_fn_mapper = {
    'gsm8k': lambda x: float(x.replace(',','').strip()),
    'svamp': lambda x: float(x.replace(',','').strip()),
    'mathqa': lambda x: x.lower().replace('"','').replace("'",'').strip(),
    'mathqa-numeric': lambda x: float(x),
    'zhouyi': lambda x: x.strip(),
}
### the answer_cot is a list of answer_cot
post_process_answer_cot_fn_mapper = {
    ('python', 'gsm8k'): lambda answer_cot: [floatify(res) for res in run_python_code(programs=answer_cot, TIMEOUT=TIMEOUT)],
    ('python', 'svamp'): lambda answer_cot: [floatify(res) for res in run_python_code(programs=answer_cot, TIMEOUT=TIMEOUT)],
    ('python', 'mathqa'): lambda answer_cot: [str(res).lower().replace('"','').replace("'",'').strip() for res in run_python_code(programs=answer_cot, TIMEOUT=TIMEOUT)],
    ('python', 'mathqa-numeric'): lambda answer_cot: [floatify(res) for res in run_python_code(programs=answer_cot, TIMEOUT=TIMEOUT)],
    ('nl', 'gsm8k'): lambda answer_cot: [floatify(res.split(answer_trigger)[-1].strip()) for res in answer_cot],
    ('nl', 'svamp'): lambda answer_cot: [floatify(res.split(answer_trigger)[-1].strip()) for res in answer_cot],
    ('nl', 'mathqa'): lambda answer_cot: [res.split(answer_trigger)[-1].lower().replace('"','').replace("'",'').strip() for res in answer_cot],
    ('nl', 'mathqa-numeric'): lambda answer_cot: [floatify(res.split(answer_trigger)[-1].strip()) for res in answer_cot],
    ('nl', 'zhouyi'): lambda answer_cot: [res.split(answer_trigger)[-1].strip() for res in answer_cot],
}
compare_answer_fn_mapper = {
    'gsm8k': lambda extracted_ans, target_answer: abs(extracted_ans - target_answer) <= 1e-2,
    'svamp': lambda extracted_ans, target_answer: abs(extracted_ans - target_answer) <= 1e-2,
    'mathqa': lambda extracted_ans, target_answer: extracted_ans == target_answer,
    'mathqa-numeric': lambda extracted_ans, target_answer: abs(extracted_ans - target_answer) <= 1e-2,
    'zhouyi': lambda extracted_ans, target_answer: extracted_ans == target_answer,
}

def prepare_datasets_and_data_loaders(args, tokenizer):
    # 确保所有进程同步开始准备数据
    # accelerator.wait_for_everyone()
    
    with accelerator.main_process_first():
        raw_dataset = DatasetDict({
            'train': Dataset.from_list(json.load(open(args['train_file'],'r'))),
            'test': Dataset.from_list(json.load(open(args['test_file'],'r'))),
        })
        accelerator.print('Raw data:', raw_dataset)
        src_name = raw_dataset['train'][0]['item_id'].split('_')[0]  # e.g., gsm8k_0, gsm8k_1, gsm8k_2, ...
        setup_cot(src_name)
        accelerator.print('Using instruction:', instruction)
        accelerator.print('Using cot_trigger:', cot_trigger)
        accelerator.print('Using answer_trigger:', answer_trigger)
        def tokenize_fn(batch, args, tokenizer):
            assert tokenizer.eos_token_id is not None, (tokenizer.eos_token_id, tokenizer.eos_token)
            new_batch = defaultdict(list)
            all_keys = list(batch.keys())
            for item_values in zip(*(batch[k] for k in all_keys)):
                item = {k: item_values[i] for i, k in enumerate(all_keys)}
                item_id, question, answer_value, answer_cot = \
                        item['item_id'], \
                        item['question'], \
                        item['answer_value'], \
                        item.get('answer_cot', None), \

                question = question.strip()
                if answer_value is not None:
                    answer_value = answer_value.strip()

                if answer_cot is not None:
                    answer_cot = answer_cot.strip()
                    if args['engine'] == 'nl':
                        answer_cot += f'{answer_trigger}{answer_value}'

                input = f'{instruction}{question}{cot_trigger}'
                output = f'{answer_cot}'
                prefix_text = f'{instruction}{question}{cot_trigger}'

                input_encode = tokenizer(input, add_special_tokens=False)
                output_encode = tokenizer(output, add_special_tokens=False)
                prefix_encode = tokenizer(prefix_text, add_special_tokens=False)

                input_ids = input_encode['input_ids'] + output_encode['input_ids'] + [tokenizer.eos_token_id]
                labels = [-100]*len(input_encode['input_ids']) + output_encode['input_ids'] + [tokenizer.eos_token_id]
                attention_mask = [1]* len(input_ids)
                prefix = prefix_encode['input_ids']
                prefix_attention_mask = prefix_encode['attention_mask']

                # Truncation
                input_ids_max_length = len(input_ids)
                # assert input_ids_max_length <= args['max_input_length'], input_ids_max_length
                input_ids = input_ids[:args['max_input_length']]
                labels = labels[:args['max_input_length']]
                attention_mask = attention_mask[:args['max_input_length']]
                prefix = prefix[:args['max_input_length']]
                prefix_attention_mask = prefix_attention_mask[:args['max_input_length']]

                ##
                new_batch['input_ids'].append(input_ids)
                new_batch['labels'].append(labels)
                new_batch['attention_mask'].append(attention_mask)
                new_batch['prefix'].append(prefix)
                new_batch['prefix_attention_mask'].append(prefix_attention_mask)
                ##
                new_batch['item_id'].append(item_id)
                new_batch['question'].append(question)
                new_batch['answer_cot'].append(answer_cot)
                new_batch['answer_value'].append(answer_value)
                new_batch['input_ids_max_length'].append(input_ids_max_length)
            
            return new_batch

        tokenized_dataset = DatasetDict({
            mode: dataset.map(
                tokenize_fn, fn_kwargs={'args': args, 'tokenizer': tokenizer}, batched=True, remove_columns=dataset.column_names, 
                num_proc=8, load_from_cache_file=False
            ) for mode, dataset in raw_dataset.items()})
        accelerator.print('Processed data:', tokenized_dataset)
        for mode, dataset in tokenized_dataset.items():
            accelerator.print(mode, f'{mode}_input_ids_max_length', max(dataset['input_ids_max_length']))

        if accelerator.is_main_process and args['wandb_log']:
            wandb.config.update({
                "src_name": src_name,
                "instruction": instruction,
                "cot_trigger": cot_trigger,
                "answer_trigger": answer_trigger,
                "raw_dataset": str(raw_dataset),
                "tokenized_dataset": str(tokenized_dataset),
                "train_input_ids_max_length": max(tokenized_dataset['train']['input_ids_max_length']),
                "test_input_ids_max_length": max(tokenized_dataset['test']['input_ids_max_length']),
            })

    def collate_fn(batch, args, tokenizer):
        max_input_length = max([len(item['input_ids']) for item in batch])
        max_target_length = max([len(item['labels']) for item in batch])
        max_prefix_length = max([len(item['prefix']) for item in batch])
        input_ids  = []
        attention_mask  = []
        labels, labels_left_padded  = [], []
        prefix_left_padded  = []
        prefix_attention_mask_left_padded  = []
        for item in batch:
            input_ids.append(item['input_ids'] + [tokenizer.pad_token_id]*(max_input_length - len(item['input_ids'])))
            attention_mask.append(item['attention_mask'] + [0]*(max_input_length - len(item['attention_mask'])))
            labels.append(item['labels'] + [-100]*(max_target_length - len(item['labels'])))

            labels_left_padded.append([-100]*(max_target_length - len(item['labels'])) + item['labels'])
            prefix_left_padded.append([tokenizer.pad_token_id]*(max_prefix_length - len(item['prefix'])) + item['prefix'])
            prefix_attention_mask_left_padded.append([0]*(max_prefix_length - len(item['prefix_attention_mask'])) + item['prefix_attention_mask'])
        forward_kwargs = {
            'input_ids': torch.LongTensor(input_ids),
            'attention_mask': torch.BoolTensor(attention_mask),
            'labels': torch.LongTensor(labels)
        }
        generate_prefix_kwargs = {
            'input_ids': torch.LongTensor(prefix_left_padded),
            'attention_mask': torch.BoolTensor(prefix_attention_mask_left_padded),
            'labels': torch.LongTensor(labels_left_padded)
        }
        return {
            'forward_kwargs': forward_kwargs,
            'generate_prefix_kwargs': generate_prefix_kwargs,
        }
    # 1. 创建分布式采样器
    train_sampler = DistributedSampler(
        tokenized_dataset['train'],
        num_replicas=accelerator.num_processes,
        rank=accelerator.process_index,
        shuffle=True
    )

    train_dataloader = DataLoader(
        tokenized_dataset['train'], 
        # sampler=train_sampler,  # 使用分布式采样器替代shuffle
        batch_size=args['batch_size'], 
        num_workers=args['num_workers'], 
        pin_memory=True, 
        collate_fn=partial(collate_fn, args=args, tokenizer=tokenizer),
        drop_last=True
    )
                        
    test_dataloader = DataLoader(tokenized_dataset['test'], shuffle=False, batch_size=args['eval_batch_size'], num_workers=args['num_workers'], pin_memory=True, 
                        collate_fn=partial(collate_fn, args=args, tokenizer=tokenizer))
    
    # accelerator.wait_for_everyone()                    
    return (tokenized_dataset['train'], train_dataloader), (tokenized_dataset['test'], test_dataloader)

def do_checkpoint(args, model, tokenizer, save_path, global_step):
    try:
        # 确保进程按顺序打印
        for rank in range(accelerator.num_processes):
            if accelerator.process_index == rank:
                accelerator.print(f"\n{'='*50}")
                accelerator.print(f"[进程 {rank} 的参数信息]")
                accelerator.print(f"{'='*50}\n")
                
                # 1. 打印总参数量
                total_params = sum(p.numel() for p in model.parameters())
                accelerator.print(f"总参数量: {total_params}")
                                    
    except Exception as e:
        accelerator.print(f"[进程 {accelerator.process_index}] 打印参数信息失败: {e}")
        return False
            
    try:
        accelerator.wait_for_everyone()
        accelerator.print(f"Rank {accelerator.process_index} do_checkpoint 同步成功")
    except Exception as e:
        accelerator.print(f"Rank {accelerator.process_index} do_checkpoint 同步失败: {e}")
        return False
    # 保存config tokenizer 
    try:
        if accelerator.is_main_process:
            config = AutoConfig.from_pretrained(args["model_name_or_path"], trust_remote_code=True)
            config.save_pretrained(args["model_dir"])
            tokenizer.save_pretrained(args["model_dir"])
    except:
        accelerator.print(f"Rank {accelerator.process_index} do_checkpoint config tokenizer 保存失败")
        return False
    try:
        if accelerator.is_main_process:
            unwrapped_model = accelerator.unwrap_model(model)
        # 打印state_dict
        accelerator.print(f"{'='*50} unwrapped_model.named_parameters()")
        for name, param in unwrapped_model.named_parameters():
            accelerator.print(f"参数 {name}: shape {param.shape}")
        accelerator.print(f"{'='*50} model.named_parameters()")
        for name, param in model.named_parameters():
            accelerator.print(f"参数 {name}: shape {param.shape}")
        accelerator.print(f"{'='*50}")
        
        accelerator.print(f"Rank {accelerator.process_index} do_checkpoint state_dict 收集成功")
    except:
        accelerator.print(f"Rank {accelerator.process_index} do_checkpoint state_dict 收集失败")
        return False
    try:        
        # if accelerator.is_main_process:
        unwrapped_model.save_pretrained(
            save_path,
            is_main_process=accelerator.is_main_process,
            save_function=accelerator.save,
            # 且分为n个模型文件
            max_shard_size="2GB",  # 添加此参数来控制分片大小
            safe_serialization=True,  # 使用安全的序列化方式
            state_dict = accelerator.get_state_dict(model)
            )
        accelerator.print('save checkpoint success!')
        accelerator.wait_for_everyone()
        return True
    except Exception as e:
        accelerator.print(f"Rank {accelerator.process_index} 保存失败: {e}")
        # accelerator.wait_for_everyone()
        return False

def train_one_epoch(args, model, train_dataset, train_dataloader, optimizer, scheduler, tokenizer,
                    global_step, test_dataset, test_dataloader, 
                    prefix, epoch, best_eval_log_dict, summary_log_dict, most_recent_ckpts_paths):
    
    model_dir = args['model_dir']
    clip_grad_norm = args.get('clip_grad_norm', None)
    evaluating_step_freq = args.get('evaluating_step_freq', None)
    logging_step_freq = args.get('logging_step_freq', None)
    saving_step_freq = args.get('saving_step_freq', None)
    model.train()
    epoch_result_dict = defaultdict(list)
    gradient_accumulation_steps = args['gradient_accumulation_steps']
    optimizer.zero_grad()
    accelerator.print(f"Rank {accelerator.process_index} 进入 train_one_epoch")
    
    # 打印train_dataloader
    accelerator.print(f"Rank {accelerator.process_index} train_dataloader: {train_dataloader}")
    # 确保在开始循环前同步RNG状态
    try:
        accelerator.wait_for_everyone()
        accelerator.print(f"Rank {accelerator.process_index} train_one_epoch 同步成功")
    except Exception as e:
        accelerator.print(f"Rank {accelerator.process_index} train_one_epoch 同步失败: {e}")
        return False
    pbar = tqdm(total=len(train_dataloader), desc=f'Train Loop [{accelerator.process_index}]')
    
    # with tqdm(enumerate(train_dataloader), total=len(train_dataloader), disable=not accelerator.is_main_process, desc='Train Loop') as t:
    for idx, batch in  enumerate(train_dataloader):
        try:
            accelerator.wait_for_everyone()
            accelerator.print(f"Rank {accelerator.process_index} train_one_epoch 第{idx}批次同步成功")
        except Exception as e:
            accelerator.print(f"Rank {accelerator.process_index} train_one_epoch 第{idx}批次同步失败: {e}")
            try:
                accelerator.wait_for_everyone()
                accelerator.print(f"Rank {accelerator.process_index} train_one_epoch 第{idx}批次重新同步成功")
            except Exception as e:
                accelerator.print(f"Rank {accelerator.process_index} train_one_epoch 第{idx}批次重新同步失败: {e}")
        try:
            # 前向传播
            output = model(**batch['forward_kwargs'])
            loss = output[0]
            
            # 缩放损失以适应梯度累积
            loss = loss / gradient_accumulation_steps
            
            # 反向传播
            accelerator.backward(loss)
            
            # 记录指标
            result_dict, extra = {}, None
            
            # 在累积足够步数后更新
            if (idx + 1) % gradient_accumulation_steps == 0:
                if clip_grad_norm is not None:
                    accelerator.clip_grad_norm_(model.parameters(), clip_grad_norm)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                
                # 更新全局步数
                global_step += 1
                
                # 记录训练损失
                epoch_result_dict['loss'].append(loss.item() * gradient_accumulation_steps)
                for k, v in result_dict.items():
                    epoch_result_dict[k].append(v)

                # 评估逻辑
                eval_log_dict = {}
                is_best = False
                if evaluating_step_freq is not None and global_step % evaluating_step_freq == 0:
                    evaluate_result_dict = {
                        f'Eval.Gen.{k}': v 
                        for k, v in evaluate_generation(args, model, test_dataset, test_dataloader, tokenizer).items()
                    }
                    eval_log_dict.update(evaluate_result_dict)
                    if eval_log_dict['Eval.Gen.value_accuracy'] > best_eval_log_dict.get('Eval.Gen.value_accuracy_best', 0):
                        is_best = True
                        best_eval_log_dict['Eval.Gen.value_accuracy_best'] = eval_log_dict['Eval.Gen.value_accuracy']
                    if 'Eval.Gen.value_accuracy' not in summary_log_dict:
                        summary_log_dict['Eval.Gen.value_accuracy'] = []
                    summary_log_dict['Eval.Gen.value_accuracy'].append(eval_log_dict['Eval.Gen.value_accuracy'])

                # 日志记录
                train_log_dict = {}
                if logging_step_freq is not None and global_step % logging_step_freq == 0:
                    train_log_dict = {
                        f'T.{k}': sum(v)/len(v) if isinstance(v, list) else v 
                        for k, v in epoch_result_dict.items()
                    }
                
                if eval_log_dict or train_log_dict:
                    log_dict = {
                        'lr': scheduler.get_last_lr()[0], 
                        **train_log_dict, 
                        **eval_log_dict, 
                        **best_eval_log_dict
                    }
                    if accelerator.is_main_process and args['wandb_log']:
                        wandb.log(log_dict, step=global_step)
                        log_dict = {'wandb': args['wandb_project'] + '|' + args['wandb_run_name'], **log_dict}
                    log_dict = {k: f'{v:.5g}' if isinstance(v, float) else v for k,v in log_dict.items()}
                    accelerator.print(f"{prefix}[E={epoch}/{args['n_epochs']}, S={global_step}] {log_dict}")

                # 保持记录数量
                for k, v in epoch_result_dict.items():
                    if len(v) > 1:
                        epoch_result_dict[k] = v[-1:]
            accelerator.print(f"Rank {accelerator.process_index} 训练批次 {idx} 成功")
        except Exception as e:
            accelerator.print(f"[进程 {accelerator.process_index}] 批次 {idx} 出错: {e}")
            accelerator.print(traceback.format_exc())
            continue
    # Metric summary:
    epoch_result_dict = {k:(sum(v)/len(v) if isinstance(v, list) else v) for k, v in epoch_result_dict.items()}
    # 更新进度条
    if accelerator.is_main_process:
        pbar.update(1)
    pbar.close()
    accelerator.print(f"Rank {accelerator.process_index} 训练epoch {epoch} 结束")
    return epoch_result_dict, global_step, evaluate_result_dict

def evaluate_generation(args, model, dataset, dataloader, tokenizer):
    return {'value_accuracy': 0}

    model.eval()
    predictions = []
    targets = []
    for idx, batch in tqdm(
        enumerate(dataloader), total=len(dataloader), disable=not accelerator.is_main_process,
        desc='Evaluation Gen Loop'):
        output_ = accelerator.unwrap_model(model).generate(
            **batch['generate_prefix_kwargs'],
            max_length=args['max_gen_length'],
            output_scores=True,
            return_dict_in_generate=True,
            num_beams=1,
            use_cache=True,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
        generated_ids = output_.sequences
        generated_ids = pad_across_processes(generated_ids, dim=1, pad_index=tokenizer.pad_token_id, pad_first=True)

        labels = batch['generate_prefix_kwargs']['labels']
        labels = pad_across_processes(labels, dim=1, pad_index=tokenizer.pad_token_id, pad_first=True)
        labels[labels == -100] = tokenizer.pad_token_id

        generated_ids, labels = accelerator.gather(generated_ids), accelerator.gather(labels)

        preds = [
            tokenizer.decode(g.cpu().numpy().tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=True).strip() for g in
            generated_ids]
        predictions.extend(preds)
        target = [
            tokenizer.decode(t.cpu().numpy().tolist(), skip_special_tokens=True, clean_up_tokenization_spaces=True).strip() for t in
            labels]
        targets.extend(target)

    predictions = predictions[:len(dataset)]
    targets = targets[:len(dataset)]

    if accelerator.is_main_process and accelerator.is_local_main_process:
        results = []
        src_name = dataset[0]['item_id'].split('_')[0]
        for pred, tar, item in zip(predictions, targets, dataset):
            cur_res = {
                'item_id': item['item_id'],
                'answer_value': item['answer_value'],
            }
            ## Processing target
            target_cot = tar.strip().split(cot_trigger)[-1].strip()
            target_value = post_process_final_answer_fn_mapper[src_name](cur_res['answer_value'])
            cur_res['target'] = target
            cur_res['target_cot'] = target_cot
            cur_res['target_value'] = target_value
            ## Processing prediction
            prediction_cot = pred.strip().split(cot_trigger)[-1].strip()
            cur_res['prediction'] = pred
            cur_res['prediction_cot'] = prediction_cot
            cur_res['prediction_value'] = None # Tobe filled
            results.append(cur_res)
        print("="*100)
        print("eval results:")
        print(results)
        execute_fn = post_process_answer_cot_fn_mapper[(args['engine'], src_name)]
        corr_value = 0
        for i, prediction_value in enumerate(execute_fn([item['prediction_cot'] for item in results])):
            target_value = results[i]['target_value']
            is_correct = compare_answer_fn_mapper[src_name](prediction_value, target_value) if prediction_value is not None else False
            results[i]['prediction_value'] = prediction_value
            results[i]['is_correct'] = is_correct
            corr_value += is_correct

        res_path = args['model_dir'].rstrip('/')+ '/' + '_res.json'
        with open(res_path, 'w') as f:
            json.dump(results, f, indent=2)

        # if args['wandb_log']:
        #     table = wandb.Table(dataframe=pd.DataFrame(results))
        #     wandb.log({"predictions": table})

        value_accuracy = corr_value / len(results) * 100
        accelerator.print(f"[Eval Info] value_accuracy: {value_accuracy:.5g}%")
        value_accuracy = torch.FloatTensor([value_accuracy]).to(accelerator.device)
    else:
        value_accuracy = torch.FloatTensor([-1.0]).to(accelerator.device)
    value_accuracy = broadcast(value_accuracy).cpu().numpy().tolist()[0]

    # Metric summary:
    model.train()
    return {'value_accuracy': value_accuracy}



In [ ]:

def training_loop(args):
    accelerator = Accelerator()
    set_seed(args['seed'] + accelerator.process_index)
    if torch.distributed.get_rank() == 0 and args['wandb_log']:
        wandb.init(project=args['wandb_project'], name=args['wandb_run_name'])
        wandb.config.update(args)
        
    tokenizer = AutoTokenizer.from_pretrained(
        args['tokenizer_name_or_path'],
        trust_remote_code=True,
        padding_side='left',  # ChatGLM 使用左侧填充
        eos_token='<|endoftext|>',  # 设置 EOS token
        pad_token='<|endoftext|>',  # 设置 PAD token
    )
    
    # 确保 tokenizer 有必要的特殊 token
    special_tokens_dict = {
        'pad_token': '<|endoftext|>',
        'eos_token': '<|endoftext|>',
        'bos_token': '<|startoftext|>',
    }
    tokenizer.add_special_tokens(special_tokens_dict)

    (train_dataset, train_dataloader), (test_dataset, test_dataloader) = prepare_datasets_and_data_loaders(args, tokenizer)
    config = AutoConfig.from_pretrained(
        args['model_name_or_path'],
        trust_remote_code=True
    )
    
    # 添加缺失的配置
    config._attn_implementation = "eager"  # 添加注意力实现方式
    
    # 加载模型
    model = AutoModelForCausalLM.from_pretrained(
        args['model_name_or_path'],
        config=config,
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
    )
    
    # 确保模型参数是 bf16 类型
    model = model.bfloat16()
    
    accelerator.print(f'[Vocab size]: {len(tokenizer)}')    
    model.resize_token_embeddings(len(tokenizer))

    if accelerator.is_main_process and args['wandb_log']:
        wandb.run.summary.update({
            'pad_token_id': tokenizer.pad_token_id,
            'eos_token_id': tokenizer.eos_token_id,
            'unk_token_id': tokenizer.unk_token_id,
            'vocab_size': len(tokenizer)
        })

    n_epochs = args['n_epochs']
    num_training_steps = (len(train_dataloader) // accelerator.num_processes * n_epochs) // args['gradient_accumulation_steps']
    warmup_step = args['warmup_step'] if args['warmup_step'] is not None and args['warmup_step'] >= 0 else int(0.1 * num_training_steps)
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in ["bias", "LayerNorm.weight"])],
            "weight_decay": args['weight_decay'],
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in ["bias", "LayerNorm.weight"])],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args['learning_rate'], eps=1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=num_training_steps)
    # scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step)
    accelerator.print(
        f"***** Running training *****\n"
        f"  Num examples = {len(train_dataset)}\n"
        f"  Num Epochs = {n_epochs}\n"
        f"  Instantaneous batch size per device = {args['batch_size']}\n"
        f"  Total train batch size (w. parallel, distributed & accumulation) = {args['batch_size']*accelerator.num_processes*args['gradient_accumulation_steps']}\n"
        f"  Total optimization steps = {num_training_steps}\n"
        f"  Warm up step: {warmup_step}\n"
        f"  Learning rate: {args['learning_rate']}\n"
    )   
    model, optimizer, train_dataloader, test_dataloader = accelerator.prepare(model, optimizer, train_dataloader, test_dataloader)
    
    global_step = 0
    evaluating_epoch_freq = args['evaluating_epoch_freq']
    logging_epoch_freq = args['logging_epoch_freq']
    saving_epoch_freq = args['saving_epoch_freq']
    model_dir=args['model_dir']
    best_eval_log_dict = {}
    summary_log_dict = {}
    os.makedirs(model_dir, exist_ok=True)
    most_recent_ckpts_paths = []
    lowest_loss = None
    with tqdm(range(1, n_epochs+1), total=n_epochs, disable=not accelerator.is_main_process) as t:
        for epoch in t:
            kwargs = {
                'args': args,
                'model': model, 
                'train_dataset': train_dataset, 
                'train_dataloader': train_dataloader, 
                'test_dataset': test_dataset,
                'test_dataloader': test_dataloader,
                'optimizer': optimizer, 
                'scheduler': scheduler,
                'global_step': global_step, 
                'tokenizer': tokenizer,
                'prefix':'', 
                'epoch': epoch,
                'best_eval_log_dict': best_eval_log_dict,
                'summary_log_dict': summary_log_dict,
                'most_recent_ckpts_paths': most_recent_ckpts_paths,
            }
            
            try:
                
                train_epoch_result_dict, global_step, evaluate_result_dict = train_one_epoch(**kwargs)
                accelerator.print(f"[进程 {accelerator.process_index}] Epoch {epoch} 训练结果: {train_epoch_result_dict}")
 
            except Exception as e:
                accelerator.print(f"[进程 {accelerator.process_index}] Epoch {epoch} 发生错误: {e}")
                break
            
            eval_log_dict = {}
            is_best = False
            
            accelerator.print(f'跳过evaluation')
            # if evaluating_epoch_freq is not None and epoch % evaluating_epoch_freq == 0:
            #     evaluate_result_dict = {f'Eval.Gen.{k}':  v for k, v in evaluate_generation(args, model, test_dataset, test_dataloader, tokenizer).items()}
            #     eval_log_dict.update(evaluate_result_dict)
            #     if eval_log_dict['Eval.Gen.value_accuracy'] > best_eval_log_dict.get('Eval.Gen.value_accuracy_best', 0):
            #         is_best = True
            #         best_eval_log_dict['Eval.Gen.value_accuracy_best'] = eval_log_dict['Eval.Gen.value_accuracy']
            #     if 'Eval.Gen.value_accuracy' not in summary_log_dict:
            #         summary_log_dict['Eval.Gen.value_accuracy'] = []
            #     summary_log_dict['Eval.Gen.value_accuracy'].append(eval_log_dict['Eval.Gen.value_accuracy'])
            if lowest_loss is None:
                lowest_loss = train_epoch_result_dict["loss"]
                is_best = True
            elif train_epoch_result_dict["loss"] < lowest_loss:
                lowest_loss = train_epoch_result_dict["loss"]
                is_best = True
                
            train_log_dict = {}
            if logging_epoch_freq is not None and epoch % logging_epoch_freq == 0:
                train_log_dict = {f'T.{k}': sum(v)/len(v) if isinstance(v, list) else v for k, v in train_epoch_result_dict.items()}

            if eval_log_dict or train_log_dict:
                log_dict = {'lr': scheduler.get_last_lr()[0], **train_log_dict, **eval_log_dict, **best_eval_log_dict}
                if accelerator.is_main_process and args['wandb_log']:
                    wandb.log(log_dict, step=global_step)
                    log_dict = {'wandb': args['wandb_project'] + '|' + args['wandb_run_name'], **log_dict}
                log_dict = {k: f'{v:.5g}' if isinstance(v, float) else v for k,v in log_dict.items()}
                accelerator.print(f"[E={epoch}/{args['n_epochs']}, S={global_step}] {log_dict}")
            
            # if saving_epoch_freq is not None and epoch % saving_epoch_freq == 0:
            if is_best:
                try:
                    accelerator.wait_for_everyone()
                    accelerator.print(f"epoch {epoch} 将开始保存权重，等待所有进程成功")
                except Exception as e:
                    accelerator.print(f"epoch {epoch} 将开始保存权重，等待所有进程失败: {e}")
                    break
                
                save_path = model_dir
                # 如果目录已存在,先清空
                if accelerator.is_main_process and os.path.exists(save_path):
                    accelerator.print(f"目录已存在, 清空目录: {save_path}")
                    # shutil.rmtree(save_path)
                os.makedirs(save_path, exist_ok=True)
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                accelerator.print(f"开始保存新的最佳checkpoint... 时间: {timestamp}")
                
                # accelerator.wait_for_everyone()
                s=do_checkpoint(args, model, tokenizer, save_path, global_step)
                # accelerator.wait_for_everyone()
                if s:
                    accelerator.print(f"保存checkpoint成功")
                else:
                    accelerator.print(f"保存checkpoint失败")
                    break


In [ ]:

if __name__ == '__main__':
    from transformers import HfArgumentParser
    NONE_INT = -100 
    NONE_STR = 'None'
    @dataclass
    class Arguments:
        model_name_or_path: str
        tokenizer_name_or_path: str
        model_dir: str
        train_file: str 
        test_file: str
        batch_size: int = field(default=4)
        eval_batch_size: int = field(default=8)
        n_epochs: int = field(default=40)
        num_workers: int = field(default=8)
        learning_rate: float = field(default=2e-5)
        weight_decay: float = field(default=1e-6)
        warmup_step: int = field(default=0)
        clip_grad_norm: float = field(default=1)
        evaluating_epoch_freq: int = field(default=1)
        logging_epoch_freq: int = field(default=1)
        saving_epoch_freq: int = field(default=1000)
        evaluating_step_freq: int = field(default=NONE_INT)
        logging_step_freq: int = field(default=NONE_INT)
        saving_step_freq: int = field(default=NONE_INT)
        seed: int = field(default=42)
        max_input_length: int = field(default=700)
        max_gen_length: int = field(default=512)
        gradient_accumulation_steps: int = field(default=1)
        keep_num_ckpt: int = field(default=1)
        # wandb stuff
        wandb_log: bool = field(default=False)
        wandb_project: str = field(default='tmp_anvfupsadfn')
        wandb_run_name: str = field(default='default_run_name')
        ###
        engine: str = field(default='nl')

    parser = HfArgumentParser(Arguments)
    (args,) = parser.parse_args_into_dataclasses()
    args = asdict(args)
    for k,v in args.items():
        if v in [NONE_INT, NONE_STR]:
            args[k] = None
    accelerator = Accelerator(gradient_accumulation_steps=args['gradient_accumulation_steps'], kwargs_handlers=[InitProcessGroupKwargs(timeout=timedelta(seconds=5*60*60))]) # wait for processing upto 5hrs
    accelerator.print(args)
    accelerator.print(json.dumps(args, indent=2, ensure_ascii=False))
    main(args)

In [ ]:
notebook_launcher(training_loop, args, num_processes=8)